### Gradient Boosted Trees with XGBoost

Jay Urbain, PhD

Credits:   
https://xgboost.readthedocs.io/en/latest/  
https://github.com/tqchen/xgboost/tree/master/demo/guide-python   
https://jessesw.com/XG-Boost/    

[XGBoost](https://github.com/dmlc/xgboost) has become incredibly popular on [Kaggle](https://github.com/dmlc/xgboost/tree/master/demo#machine-learning-challenge-winning-solutions) for any problems dealing with structured (tabular) data. 

scikit-learn implementation of Gradient Boosted Trees: [sklearn's version](http://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting).

There are several technical issues why you may want to consider the xgboost implementation. The most significant is speed.

Reference [this paper](http://www.jmlr.org/proceedings/papers/v42/chen14.pdf) on XGBoost's first major debut in the [Higgs-Boson Kaggle competition](https://www.kaggle.com/c/higgs-boson) that XGBoost has a nice speed advantage over sklearn. 


Szilard Pafka has done an [excellent benchmark](https://github.com/szilard/benchm-ml) of a variety of machine learning libraries and he also found XGBoost to be much faster. 

What about XGBoost makes it faster? Gradient boosted trees, have to be built in series so that a step of gradient descent can be taken in order to minimize a loss function. Unlike Random Forests, you can't simply build the trees in parallel. XGBoost, however, builds the tree **itself** in a parallel fashion. More details about this are available in the XGBoost paper by Chen and Guestrin [here](https://arxiv.org/pdf/1603.02754v1.pdf). Essentially the information contained in each feature column can have statistics calculated on it in parallel.

What you get in return for this speed is a much faster grid search for optimizing hyperparameters in model tuning. 

#### Getting and Preprocessing the Data

For comparative purposes, we will work with the ["Adult Data Set"](https://archive.ics.uci.edu/ml/datasets/Adult) used in our LogisticRegression lab.

Our task is to predict whether a certain individual has an income of greater than 50,000 based on their demographic information. In the dataset description found [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names), we can see that the best model they came up with at the time had an accuracy of 85.95% (14.05% error on the test set). 

First, load in numpy and pandas, and download the data, which is already split into train/test sets. Skip the header row.

In [1]:
import numpy as np
import pandas as pd

Load in the same train and test sets used by the original authors.

In [2]:
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
                      skiprows = 1, header = None) # Make sure to skip a row for the test set

Check the datasets.

In [3]:
train_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
test_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


A few potential issues:

- We don't have a column header for our data.
- There seem to be some unknown values in the fifth row of the test set (the question marks). 
- The target values have periods at the end in the test set but do not in the training set (<=50K. vs. <=50K).

Based on the accompanying dataset description, we can see the column names. Add the column names to the train and test first.

In [5]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']

Apply names to both dataframes. 

In [6]:
train_set.columns = col_labels
test_set.columns = col_labels

Next, let's check to see if pandas has identified any of these missing values. 

In [7]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [8]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
age               16281 non-null int64
workclass         16281 non-null object
fnlwgt            16281 non-null int64
education         16281 non-null object
education_num     16281 non-null int64
marital_status    16281 non-null object
occupation        16281 non-null object
relationship      16281 non-null object
race              16281 non-null object
sex               16281 non-null object
capital_gain      16281 non-null int64
capital_loss      16281 non-null int64
hours_per_week    16281 non-null int64
native_country    16281 non-null object
wage_class        16281 non-null object
dtypes: int64(6), object(9)
memory usage: 1.9+ MB


According to the accompanying notes, the original datasets had 32561 in train and 16281 with test. However, unknowns are included and have been labeled with a question mark (?). The test results documented were done after removing all unknowns. Therefore, to see if we can beat their best results, we need to remove the same unknown rows. 

If we do so, we should have 30162 in train and 15060 in test. Let's see if we can remove all of these unknown rows. 

It turns out the question mark was actually entered with a space first. Let's do a simple test to see what would happen if we dropped all rows that contain an unknown marked with a ' ?'.

In [9]:
train_set.replace(' ?', np.nan).dropna().shape 

(30162, 15)

In [10]:
test_set.replace(' ?', np.nan).dropna().shape

(15060, 15)

These must be our missing rows since the numbers add up now if we drop them. Let's apply this change to our test and training sets.

In [11]:
train_nomissing = train_set.replace(' ?', np.nan).dropna()
test_nomissing = test_set.replace(' ?', np.nan).dropna()

Now that we have taken care of the missing value problem, we still have an issue with the target income thresholds being encoded slightly differently in test vs. train. 

We need these to match up appropriately, so we are going to need to fix either the test or training set to make them match up. 

Replace all of the '<=50K.' with '<=50K' and the same for '>50K.' with '>50K', we are just dropping the periods. 

We can use the replace method from pandas to fix this. 

In [12]:
test_nomissing['wage_class'] = test_nomissing.wage_class.replace({' <=50K.': ' <=50K', ' >50K.':' >50K'})

Checking the unique values from each set, we can see if they now match.

In [13]:
test_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

In [14]:
train_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

Now we need to make sure everything encoded as a string is turned into a variable. 

#### Applying  Encoding to Categoricals

There is some debate as to whether you are technically required to use one-hot encoding for categorical features (since there isn't supposed to be any inherent ordering). [This discussion](https://github.com/szilard/benchm-ml/issues/1) on the subject interesting. 

Essentially, one-hot encoding isn't necessary for tree-based algorithms, but you may run into a problem using this on linear models.  

First, combine them together into a single dataset. 

In [15]:
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically

In [16]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null object
fnlwgt            45222 non-null int64
education         45222 non-null object
education_num     45222 non-null int64
marital_status    45222 non-null object
occupation        45222 non-null object
relationship      45222 non-null object
race              45222 non-null object
sex               45222 non-null object
capital_gain      45222 non-null int64
capital_loss      45222 non-null int64
hours_per_week    45222 non-null int64
native_country    45222 non-null object
wage_class        45222 non-null object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


If the feature is not already numerical, we need to encode it as one. We can use pandas Categorical codes for this task. 

In [17]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [18]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null int8
fnlwgt            45222 non-null int64
education         45222 non-null int8
education_num     45222 non-null int64
marital_status    45222 non-null int8
occupation        45222 non-null int8
relationship      45222 non-null int8
race              45222 non-null int8
sex               45222 non-null int8
capital_gain      45222 non-null int64
capital_loss      45222 non-null int64
hours_per_week    45222 non-null int64
native_country    45222 non-null int8
wage_class        45222 non-null int8
dtypes: int64(6), int8(9)
memory usage: 2.8 MB


Now that we have all of our features encoded, we need to split these back into their original train/test sizes. Since they haven't been shuffled we just need to retrieve the same indices as before. 

In [19]:
final_train = combined_set[:train_nomissing.shape[0]] # Up to the last initial training set row
final_test = combined_set[train_nomissing.shape[0]:]  # Past the last initial training set row

#### Initial Model Setup and Grid Search

Remove the target value inside the train and test frames prior to feeding into XGBoost.

In [20]:
y_train = final_train.pop('wage_class')
y_test = final_test.pop('wage_class')

Import the libraries we will need to do grid search for XGBoost. We can use sklearn's API. 

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

To review the parameters that can be tuned in XGBoost, have a look at the [parameter documentation](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst). 

A useful example is available [here](https://github.com/dmlc/xgboost/blob/master/demo/guide-python/custom_objective.py).

In [29]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                             cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1,
                             return_train_score=True) 
# Optimize for accuracy since that is the metric used in the Adult Data Set notation

Now let's run our grid search with 5-fold cross-validation and see which parameters perform the best.

In [30]:
optimized_GBM.fit(final_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

Let's check our grid scores.

In [31]:
optimized_GBM.cv_results_

{'mean_fit_time': array([38.05417356, 37.84070706, 36.05836725, 49.30986862, 48.47288327,
        47.06869841, 63.41675344, 58.68249044, 47.37163076]),
 'std_fit_time': array([0.22499025, 0.04537635, 1.41610977, 0.08346701, 0.43032541,
        0.22646438, 0.15052846, 2.39786558, 5.08487728]),
 'mean_score_time': array([0.33268046, 0.31883764, 0.30720892, 0.65682526, 0.55268512,
        0.5184598 , 1.16563668, 0.67396722, 0.46826453]),
 'std_score_time': array([0.01196734, 0.00549443, 0.02788719, 0.021732  , 0.04582518,
        0.00964167, 0.01760205, 0.0733309 , 0.05362652]),
 'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_min_child_weight': masked_array(data=[1, 3, 5, 1, 3, 5, 1, 3, 5],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_val

The first hyperparameter combination performed best and we've already exceeded our target of 85.95% accuracy in our cross-validation. 

We can try optimizing some other hyperparameters now to see if we can beat a mean of 86.78% accuracy. This time, we will play around with subsampling along with lowering the learning rate to see if that helps.

In [ ]:
cv_params = {'learning_rate': [0.1, 0.01], 'subsample': [0.7,0.8,0.9]}
ind_params = {'n_estimators': 1000, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth': 3, 'min_child_weight': 1}


optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)
optimized_GBM.fit(final_train, y_train)

Again, check the grid scores. 

In [ ]:
optimized_GBM.grid_scores_

No real improvement. We may be able to optimize a little further by utilizing XGBoost's built-in cv which allows early stopping to prevent overfitting. 

#### Early stopping CV

Based on the CV testing performed earlier, we want to utilize the following parameters:

- Learning_rate (eta) = 0.1
- Subsample, colsample_bytree = 0.8
- Max_depth = 3
- Min_child_weight = 1

There are a few other parameters we could tune in theory to squeeze out a little more performance, but this is a good enough starting point. 

To increase the performance of XGBoost's speed through many iterations of the training set, we can create a DMatrix. The DMatrix sorts the data to optimize for XGBoost when it builds trees, making the algorithm more efficient. To create a DMatrix:

In [ ]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient

Now let's specify our parameters (with slightly different syntax in some places for the XGBoost native API) and set our stopping criteria. For now, let's be aggressive with the stopping and say we don't want the accuracy to improve for at least 100 new trees.

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

We can look at our CV results to see how accurate we were with these settings. The output is automatically saved into a pandas dataframe for us. 

In [ ]:
cv_xgb.tail(5)

Our CV test error at this number of iterations is 12.897%,  or 87.103% accuracy. 

Now that we have our best settings, let's create this as an XGBoost object model that we can reference later. 

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)

Notice that it trains very quickly. With our XG model object, we can then plot our feature importances using a built-in method. This is similar to the feature importances found in sklearn. 

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1.5)

In [ ]:
xgb.plot_importance(final_gb)

This will tell us which features were most important in the series of trees. The 'fnlwgt' feature seems to have the most importance. Filing capital gains was also important, which makes sense given that only those with greater incomes have the ability to invest. Race and sex were not as important. This may be because we are just predicting a specific threshold (below/above 50000 a year) instead of a precise income. 

If the built-in feature importance method isn't what you wanted, you can make your own chart of feature importances using the get_fscore method:

In [ ]:
importances = final_gb.get_fscore()
importances

Given this dictionary, we can now make a plot of it ourselves. 

In [ ]:
importance_frame = pd.DataFrame({'Importance': list(importances.values()), 'Feature': list(importances.keys())})
importance_frame.sort_values(by = 'Importance', inplace = True)
importance_frame.plot(kind = 'barh', x = 'Feature', figsize = (8,8), color = 'orange')

Now that we have an understanding of the feature importances, we can better understand what is driving the splits most for the trees and where we may be able to make some improvements in feature engineering if possible. 

#### Analyzing Performance on Test Data

The model has now been tuned using cross-validation grid search through the sklearn API and early stopping through the built-in XGBoost API. Now, we can see how it finally performs on the test set. Does it match our CV performance? First, create another DMatrix (this time for the test data). 

In [ ]:
testdmat = xgb.DMatrix(final_test)

Now let's use sklearn's accuracy metric to see how well we did on the test set. 

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

You can see that the predict function for XGBoost outputs probabilities by default and not actual class labels. To calculate accuracy we need to convert these to a 0/1 label. We will set 0.5 probability as our threshold. 

In [ ]:
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred

Now we can calculate our accuracy. 

In [ ]:
accuracy_score(y_pred, y_test), 1-accuracy_score(y_pred, y_test)

Our final accuracy is 86.94%, or a 13.05% error rate. We beat our goal by a one percentage point! 

#### Further improvement

- Make sure to read the [documentation](https://xgboost.readthedocs.org/en/latest/) thoroughly if you want to utilize some of its more advanced features
- This is a rapidly evolving library, so additional features may be introduced that affect its functionality
- XGBoost is available in most of the other common languages for data science as well such as Julia and R if those are preferable
- XGBoost can also now be utilized at scale, running on Spark and Flink with the recently released [XGBoost4J](http://dmlc.ml/2016/03/14/xgboost4j-portable-distributed-xgboost-in-spark-flink-and-dataflow.html)

Some other helpful information:

- This [YouTube video](https://www.youtube.com/watch?v=ufHo8vbk6g4) given by Tong He on XGBoost in detail. This describes the R version, but the API is very similar and there is a lot of great information in here on how XGBoost actually works.
- Here is his [slide deck](http://www.slideshare.net/ShangxuanZhang/xgboost) that covers the same material.
- Another tutorial guide on hyperparameter tuning from Aarshay Jain [here](http://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)
